In [ ]:
import numpy as np
import math
import time
import torch
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from Modules.training_functions import *
from Modules.pennylane_functions import *

# if gpu available, set device to gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU")
else:
    device = torch.device("cpu")
    print("WARNING: Could not find GPU, using the CPU")
T=25
all=np.load(f'Data/dataset_ld_{ld_dim}_{9}.npy')
all=all[:200]
for i in range(9):
    x=np.load(f'Data/dataset_ld_{ld_dim}_{i}.npy')
    all=np.concatenate((x[:200],all))

mnist_images=all

np.random.shuffle(mnist_images)
mnist_images = torch.tensor(mnist_images).to(device)

# make dataloader
data_loader = torch.utils.data.DataLoader(mnist_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
qc_array=np.array([0,64,96,112,120,124])
min_array=np.array([0.1,0.05,0.01,0.08])
layer_array=np.array([5,10,20,50])
print(NUM_QUBITS)
print(T)
zero = torch.zeros(BATCH_SIZE, 2**NUM_QUBITS-ld_dim).to(device)
for layer_indx in range(len(layer_array)):
    n_layer=layer_array[layer_indx]
    for q_indx in range(len(qc_array)):
        qc=qc_array[q_indx]
        for min_indx in range(len(min_array)):
            min_b=min_array[min_indx]

            betas      = np.insert(np.linspace(10e-8,min_b, T), 0, 0)
            print(np.shape(betas))
            alphas     = 1 - betas
            alphas_bar = np.cumprod(alphas)
            pi         = math.pi
            betas      = torch.tensor(betas).float().to(device)
            alphas     = torch.tensor(alphas).float().to(device)
            alphas_bar = torch.tensor(alphas_bar).float().to(device)
            theta_1    = Variable(torch.rand((n_layer*3*NUM_QUBITS+n_layer*3*(NUM_QUBITS)), device = device), requires_grad=True)
            optimizer = torch.optim.Adam([theta_1], lr = LEARNING_RATE)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = SCHEDULER_PATIENCE, gamma = SCHEDULER_GAMMA, verbose = False)
            trained_thetas_1 = []
            loss_history = []
            best_loss = 1e10

            for epoch in range(NUM_EPOCHS):
                print(epoch)

                t0 = time.time()
                num_batch=0
                tot_loss=0

                for image_batch in data_loader:

                    # extract batch of random times and betas
                    t = torch.randint(0, T, size = (BATCH_SIZE, ), device=device)
                    betas_batch = betas[t].to(device)
                    alphas_batch=alphas_bar[t].to(device)

                    # assemble input at t add noise (t+1)
                    target_batch = assemble_input(image_batch, t, alphas_bar,ld_dim ,device)
                    input_batch  = noise_step(target_batch, t+1, betas,ld_dim, device)
                    target_batch = target_batch / torch.norm(target_batch, dim = 1).view(-1, 1)
                    input_batch  = input_batch / torch.norm(input_batch, dim = 1).view(-1, 1)
                    

                    # concatenate the two tensors along the second dimension
                    input_batch = torch.cat((input_batch, zero), dim=1)
                    target_batch = torch.cat((target_batch, zero), dim=1)
                    # Feed to circuit, compute the loss and update the weights
                    num_batch+=1
                    loss = loss_fn_aq(qc,theta_1,n_layer, input_batch, target_batch)
                    tot_loss+=loss.item()
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                # append parameters and print loss
                trained_thetas_1.append(theta_1.cpu().clone().detach().numpy())

                loss_history.append(tot_loss/num_batch)
                if loss.item()< best_loss:
                    best_loss=loss.item()

                # implement learning rate scheduler
                scheduler.step()


            # print every epoch
                print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={((time.time()-t0)*(NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))/60:.2f} min{min_b} nl{n_layer} QC{qc}')
                #print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={(((NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))):.2f} min{min_b} nl{n_layer}')
                
            np.save(f'thetas_T{T}_nl{n_layer}_min{min_b}_qc{qc}_{Q_ANCILLA}_ld{ld_dim}.npy',trained_thetas_1)
            np.save(f'loss__T{T}_nl{n_layer}_min{min_b}_qc{qc}_ancilla{Q_ANCILLA}_ld{ld_dim}.npy',loss_history)

(2000, 8)
7
25
(26,)
0
T=25 Epoch: 1/50 - Loss: 0.7772 b_loss=0.7772 - T: 3.20s/epoch ,tempo_previto=256.30 min0.1 nl5 QC0
1
T=25 Epoch: 2/50 - Loss: 0.6691 b_loss=0.6691 - T: 3.30s/epoch ,tempo_previto=264.14 min0.1 nl5 QC0
2
T=25 Epoch: 3/50 - Loss: 0.5406 b_loss=0.5406 - T: 3.48s/epoch ,tempo_previto=278.60 min0.1 nl5 QC0
3
T=25 Epoch: 4/50 - Loss: 0.4629 b_loss=0.4629 - T: 3.37s/epoch ,tempo_previto=268.98 min0.1 nl5 QC0
4
T=25 Epoch: 5/50 - Loss: 0.4087 b_loss=0.4087 - T: 3.45s/epoch ,tempo_previto=276.09 min0.1 nl5 QC0
5
T=25 Epoch: 6/50 - Loss: 0.3281 b_loss=0.3281 - T: 3.36s/epoch ,tempo_previto=268.15 min0.1 nl5 QC0
6
T=25 Epoch: 7/50 - Loss: 0.3211 b_loss=0.3211 - T: 3.55s/epoch ,tempo_previto=283.47 min0.1 nl5 QC0
7
T=25 Epoch: 8/50 - Loss: 0.2815 b_loss=0.2815 - T: 3.37s/epoch ,tempo_previto=269.23 min0.1 nl5 QC0
8
T=25 Epoch: 9/50 - Loss: 0.2601 b_loss=0.2601 - T: 3.43s/epoch ,tempo_previto=274.11 min0.1 nl5 QC0
9
T=25 Epoch: 10/50 - Loss: 0.2350 b_loss=0.2350 - T: 3.46s/e

T=25 Epoch: 30/50 - Loss: 0.0970 b_loss=0.0934 - T: 3.61s/epoch ,tempo_previto=268.82 min0.05 nl5 QC0
30
T=25 Epoch: 31/50 - Loss: 0.1039 b_loss=0.0934 - T: 3.79s/epoch ,tempo_previto=282.65 min0.05 nl5 QC0
31
T=25 Epoch: 32/50 - Loss: 0.0998 b_loss=0.0934 - T: 3.76s/epoch ,tempo_previto=280.21 min0.05 nl5 QC0
32
T=25 Epoch: 33/50 - Loss: 0.0985 b_loss=0.0934 - T: 3.89s/epoch ,tempo_previto=289.88 min0.05 nl5 QC0
33
T=25 Epoch: 34/50 - Loss: 0.0954 b_loss=0.0934 - T: 3.87s/epoch ,tempo_previto=288.40 min0.05 nl5 QC0
34
T=25 Epoch: 35/50 - Loss: 0.1046 b_loss=0.0934 - T: 4.04s/epoch ,tempo_previto=300.51 min0.05 nl5 QC0
35
T=25 Epoch: 36/50 - Loss: 0.1003 b_loss=0.0934 - T: 4.03s/epoch ,tempo_previto=299.70 min0.05 nl5 QC0
36
T=25 Epoch: 37/50 - Loss: 0.1073 b_loss=0.0934 - T: 4.03s/epoch ,tempo_previto=299.63 min0.05 nl5 QC0
37
T=25 Epoch: 38/50 - Loss: 0.0998 b_loss=0.0934 - T: 4.10s/epoch ,tempo_previto=305.20 min0.05 nl5 QC0
38
T=25 Epoch: 39/50 - Loss: 0.0972 b_loss=0.0934 - T: 4.1

T=25 Epoch: 9/50 - Loss: 0.1722 b_loss=0.1722 - T: 4.55s/epoch ,tempo_previto=295.19 min0.08 nl5 QC0
9
T=25 Epoch: 10/50 - Loss: 0.1566 b_loss=0.1566 - T: 4.55s/epoch ,tempo_previto=295.18 min0.08 nl5 QC0
10
T=25 Epoch: 11/50 - Loss: 0.1381 b_loss=0.1381 - T: 4.58s/epoch ,tempo_previto=296.97 min0.08 nl5 QC0
11
T=25 Epoch: 12/50 - Loss: 0.1254 b_loss=0.1254 - T: 4.77s/epoch ,tempo_previto=308.98 min0.08 nl5 QC0
12
T=25 Epoch: 13/50 - Loss: 0.1222 b_loss=0.1222 - T: 4.70s/epoch ,tempo_previto=304.64 min0.08 nl5 QC0
13
T=25 Epoch: 14/50 - Loss: 0.1110 b_loss=0.1110 - T: 4.77s/epoch ,tempo_previto=309.21 min0.08 nl5 QC0
14
T=25 Epoch: 15/50 - Loss: 0.1043 b_loss=0.1043 - T: 4.81s/epoch ,tempo_previto=311.46 min0.08 nl5 QC0
15
T=25 Epoch: 16/50 - Loss: 0.1051 b_loss=0.1043 - T: 5.05s/epoch ,tempo_previto=327.00 min0.08 nl5 QC0
16
T=25 Epoch: 17/50 - Loss: 0.1004 b_loss=0.1004 - T: 5.02s/epoch ,tempo_previto=324.72 min0.08 nl5 QC0
17
T=25 Epoch: 18/50 - Loss: 0.0925 b_loss=0.0925 - T: 5.06s

T=25 Epoch: 38/50 - Loss: 0.0601 b_loss=0.0601 - T: 5.07s/epoch ,tempo_previto=398.43 min0.1 nl5 QC64
38
T=25 Epoch: 39/50 - Loss: 0.0628 b_loss=0.0601 - T: 5.21s/epoch ,tempo_previto=408.76 min0.1 nl5 QC64
39
T=25 Epoch: 40/50 - Loss: 0.0626 b_loss=0.0601 - T: 5.38s/epoch ,tempo_previto=422.14 min0.1 nl5 QC64
40
T=25 Epoch: 41/50 - Loss: 0.0650 b_loss=0.0601 - T: 5.16s/epoch ,tempo_previto=404.83 min0.1 nl5 QC64
41
T=25 Epoch: 42/50 - Loss: 0.0631 b_loss=0.0601 - T: 6.19s/epoch ,tempo_previto=485.74 min0.1 nl5 QC64
42
T=25 Epoch: 43/50 - Loss: 0.0676 b_loss=0.0601 - T: 3.47s/epoch ,tempo_previto=272.36 min0.1 nl5 QC64
43
T=25 Epoch: 44/50 - Loss: 0.0571 b_loss=0.0571 - T: 3.49s/epoch ,tempo_previto=274.10 min0.1 nl5 QC64
44
T=25 Epoch: 45/50 - Loss: 0.0628 b_loss=0.0571 - T: 3.52s/epoch ,tempo_previto=275.68 min0.1 nl5 QC64
45
T=25 Epoch: 46/50 - Loss: 0.0637 b_loss=0.0571 - T: 3.60s/epoch ,tempo_previto=282.11 min0.1 nl5 QC64
46
T=25 Epoch: 47/50 - Loss: 0.0654 b_loss=0.0571 - T: 3.4

T=25 Epoch: 16/50 - Loss: 0.0711 b_loss=0.0711 - T: 3.60s/epoch ,tempo_previto=248.07 min0.01 nl5 QC64
16
T=25 Epoch: 17/50 - Loss: 0.0730 b_loss=0.0711 - T: 3.61s/epoch ,tempo_previto=248.83 min0.01 nl5 QC64
17
T=25 Epoch: 18/50 - Loss: 0.0715 b_loss=0.0711 - T: 3.68s/epoch ,tempo_previto=253.65 min0.01 nl5 QC64
18
T=25 Epoch: 19/50 - Loss: 0.0680 b_loss=0.0680 - T: 3.75s/epoch ,tempo_previto=258.05 min0.01 nl5 QC64
19
T=25 Epoch: 20/50 - Loss: 0.0723 b_loss=0.0680 - T: 3.94s/epoch ,tempo_previto=271.03 min0.01 nl5 QC64
20
T=25 Epoch: 21/50 - Loss: 0.0705 b_loss=0.0680 - T: 3.93s/epoch ,tempo_previto=270.48 min0.01 nl5 QC64
21
T=25 Epoch: 22/50 - Loss: 0.0708 b_loss=0.0680 - T: 4.05s/epoch ,tempo_previto=278.65 min0.01 nl5 QC64
22
T=25 Epoch: 23/50 - Loss: 0.0702 b_loss=0.0680 - T: 4.11s/epoch ,tempo_previto=283.04 min0.01 nl5 QC64
23
T=25 Epoch: 24/50 - Loss: 0.0698 b_loss=0.0680 - T: 4.11s/epoch ,tempo_previto=282.91 min0.01 nl5 QC64
24
T=25 Epoch: 25/50 - Loss: 0.0633 b_loss=0.0633

...

T=25 Epoch: 44/50 - Loss: 0.0021 b_loss=0.0019 - T: 14.90s/epoch ,tempo_previto=411.11 min0.01 nl20 QC96
44
T=25 Epoch: 45/50 - Loss: 0.0019 b_loss=0.0019 - T: 13.19s/epoch ,tempo_previto=363.84 min0.01 nl20 QC96
45
T=25 Epoch: 46/50 - Loss: 0.0018 b_loss=0.0018 - T: 15.40s/epoch ,tempo_previto=424.41 min0.01 nl20 QC96
46
T=25 Epoch: 47/50 - Loss: 0.0021 b_loss=0.0018 - T: 15.36s/epoch ,tempo_previto=423.17 min0.01 nl20 QC96
47
T=25 Epoch: 48/50 - Loss: 0.0020 b_loss=0.0018 - T: 14.50s/epoch ,tempo_previto=399.16 min0.01 nl20 QC96
48
T=25 Epoch: 49/50 - Loss: 0.0021 b_loss=0.0018 - T: 13.41s/epoch ,tempo_previto=369.03 min0.01 nl20 QC96
49
T=25 Epoch: 50/50 - Loss: 0.0021 b_loss=0.0018 - T: 13.64s/epoch ,tempo_previto=374.98 min0.01 nl20 QC96
(26,)
0
T=25 Epoch: 1/50 - Loss: 0.4047 b_loss=0.4047 - T: 14.08s/epoch ,tempo_previto=328.19 min0.08 nl20 QC96
1
T=25 Epoch: 2/50 - Loss: 0.1827 b_loss=0.1827 - T: 14.53s/epoch ,tempo_previto=338.55 min0.08 nl20 QC96
2
T=25 Epoch: 3/50 - Loss: 0.

T=25 Epoch: 21/50 - Loss: 0.0167 b_loss=0.0147 - T: 15.12s/epoch ,tempo_previto=561.84 min0.1 nl20 QC112
21
T=25 Epoch: 22/50 - Loss: 0.0150 b_loss=0.0147 - T: 13.93s/epoch ,tempo_previto=517.18 min0.1 nl20 QC112
22
T=25 Epoch: 23/50 - Loss: 0.0157 b_loss=0.0147 - T: 14.56s/epoch ,tempo_previto=540.38 min0.1 nl20 QC112
23
T=25 Epoch: 24/50 - Loss: 0.0158 b_loss=0.0147 - T: 15.71s/epoch ,tempo_previto=582.84 min0.1 nl20 QC112
24
T=25 Epoch: 25/50 - Loss: 0.0159 b_loss=0.0147 - T: 15.36s/epoch ,tempo_previto=569.52 min0.1 nl20 QC112
25
T=25 Epoch: 26/50 - Loss: 0.0167 b_loss=0.0147 - T: 12.64s/epoch ,tempo_previto=468.64 min0.1 nl20 QC112
26
T=25 Epoch: 27/50 - Loss: 0.0164 b_loss=0.0147 - T: 14.49s/epoch ,tempo_previto=536.86 min0.1 nl20 QC112
27
T=25 Epoch: 28/50 - Loss: 0.0163 b_loss=0.0147 - T: 13.73s/epoch ,tempo_previto=508.29 min0.1 nl20 QC112
28
T=25 Epoch: 29/50 - Loss: 0.0162 b_loss=0.0147 - T: 14.71s/epoch ,tempo_previto=544.47 min0.1 nl20 QC112
29
T=25 Epoch: 30/50 - Loss: 0.

T=25 Epoch: 47/50 - Loss: 0.0068 b_loss=0.0057 - T: 21.51s/epoch ,tempo_previto=682.15 min0.05 nl20 QC112
47
T=25 Epoch: 48/50 - Loss: 0.0065 b_loss=0.0057 - T: 21.94s/epoch ,tempo_previto=695.38 min0.05 nl20 QC112
48
T=25 Epoch: 49/50 - Loss: 0.0065 b_loss=0.0057 - T: 23.76s/epoch ,tempo_previto=752.81 min0.05 nl20 QC112
49
T=25 Epoch: 50/50 - Loss: 0.0071 b_loss=0.0057 - T: 16.13s/epoch ,tempo_previto=510.91 min0.05 nl20 QC112
(26,)
0
T=25 Epoch: 1/50 - Loss: 0.3615 b_loss=0.3615 - T: 14.21s/epoch ,tempo_previto=390.68 min0.01 nl20 QC112
1
T=25 Epoch: 2/50 - Loss: 0.1599 b_loss=0.1599 - T: 14.07s/epoch ,tempo_previto=386.55 min0.01 nl20 QC112
2
T=25 Epoch: 3/50 - Loss: 0.1042 b_loss=0.1042 - T: 14.44s/epoch ,tempo_previto=396.31 min0.01 nl20 QC112
3
T=25 Epoch: 4/50 - Loss: 0.0771 b_loss=0.0771 - T: 16.35s/epoch ,tempo_previto=448.54 min0.01 nl20 QC112
4
T=25 Epoch: 5/50 - Loss: 0.0548 b_loss=0.0548 - T: 17.21s/epoch ,tempo_previto=471.81 min0.01 nl20 QC112
5
T=25 Epoch: 6/50 - Loss:

T=25 Epoch: 23/50 - Loss: 0.0115 b_loss=0.0115 - T: 14.70s/epoch ,tempo_previto=325.13 min0.08 nl20 QC112
23
T=25 Epoch: 24/50 - Loss: 0.0125 b_loss=0.0115 - T: 13.35s/epoch ,tempo_previto=295.00 min0.08 nl20 QC112
24
T=25 Epoch: 25/50 - Loss: 0.0115 b_loss=0.0115 - T: 12.79s/epoch ,tempo_previto=282.49 min0.08 nl20 QC112
25
T=25 Epoch: 26/50 - Loss: 0.0114 b_loss=0.0114 - T: 14.43s/epoch ,tempo_previto=318.37 min0.08 nl20 QC112
26
T=25 Epoch: 27/50 - Loss: 0.0122 b_loss=0.0114 - T: 14.34s/epoch ,tempo_previto=316.17 min0.08 nl20 QC112
27
T=25 Epoch: 28/50 - Loss: 0.0127 b_loss=0.0114 - T: 13.12s/epoch ,tempo_previto=289.14 min0.08 nl20 QC112
28
T=25 Epoch: 29/50 - Loss: 0.0131 b_loss=0.0114 - T: 13.74s/epoch ,tempo_previto=302.43 min0.08 nl20 QC112
29
T=25 Epoch: 30/50 - Loss: 0.0126 b_loss=0.0114 - T: 13.17s/epoch ,tempo_previto=289.82 min0.08 nl20 QC112
30
T=25 Epoch: 31/50 - Loss: 0.0133 b_loss=0.0114 - T: 13.79s/epoch ,tempo_previto=303.20 min0.08 nl20 QC112
31
T=25 Epoch: 32/50 -

T=25 Epoch: 49/50 - Loss: 0.0177 b_loss=0.0147 - T: 14.63s/epoch ,tempo_previto=524.55 min0.1 nl20 QC120
49
T=25 Epoch: 50/50 - Loss: 0.0144 b_loss=0.0144 - T: 13.96s/epoch ,tempo_previto=500.33 min0.1 nl20 QC120
(26,)
0
T=25 Epoch: 1/50 - Loss: 0.4352 b_loss=0.4352 - T: 14.61s/epoch ,tempo_previto=462.47 min0.05 nl20 QC120
1
T=25 Epoch: 2/50 - Loss: 0.2685 b_loss=0.2685 - T: 13.86s/epoch ,tempo_previto=438.53 min0.05 nl20 QC120
2
T=25 Epoch: 3/50 - Loss: 0.1941 b_loss=0.1941 - T: 14.23s/epoch ,tempo_previto=449.90 min0.05 nl20 QC120
3
T=25 Epoch: 4/50 - Loss: 0.1375 b_loss=0.1375 - T: 14.12s/epoch ,tempo_previto=446.23 min0.05 nl20 QC120
4
T=25 Epoch: 5/50 - Loss: 0.1046 b_loss=0.1046 - T: 12.88s/epoch ,tempo_previto=406.87 min0.05 nl20 QC120
5
T=25 Epoch: 6/50 - Loss: 0.0756 b_loss=0.0756 - T: 14.78s/epoch ,tempo_previto=466.51 min0.05 nl20 QC120
6
T=25 Epoch: 7/50 - Loss: 0.0568 b_loss=0.0568 - T: 16.04s/epoch ,tempo_previto=506.13 min0.05 nl20 QC120
7
T=25 Epoch: 8/50 - Loss: 0.040